In [1]:
import numpy as np
import matplotlib.pyplot as plt

# HOD parameters
log10_Mmin = 13.05
log10_M1 = 14.58
alpha = 1.67
sigma_logM = 0.2  # typical value for the width of the central galaxy transition

# Mass array
log10_M = np.linspace(11, 15.5, 500)
M = 10**log10_M

# HOD components
def N_central(M, Mmin, sigma_logM):
    return 0.5 * (1 + np.erf((np.log10(M) - np.log10(Mmin)) / sigma_logM))

def N_satellite(M, M1, alpha, N_cen):
    return N_cen * (M / M1)**alpha * (M > M1)

# Calculate HOD
Mmin = 10**log10_Mmin
M1_val = 10**log10_M1
N_cen = N_central(M, Mmin, sigma_logM)
N_sat = N_satellite(M, M1_val, alpha, N_cen)
N_total = N_cen + N_sat

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(log10_M, N_cen, label=r'Central galaxies', color='blue')
plt.plot(log10_M, N_sat, label=r'Satellite galaxies', color='green')
plt.plot(log10_M, N_total, label=r'Total $\langle N_{\mathrm{gal}} \rangle$', color='black', linestyle='--')

plt.xlabel(r'$\log_{10}(M_{\mathrm{halo}}/M_\odot)$')
plt.ylabel(r'$\langle N_{\mathrm{gal}} \rangle$')
plt.title('Halo Occupation Distribution (HOD)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


AttributeError: module 'numpy' has no attribute 'erf'

In [ ]:
import scipy.special as sp

# Redefine N_central using scipy's erf
def N_central(M, Mmin, sigma_logM):
    return 0.5 * (1 + sp.erf((np.log10(M) - np.log10(Mmin)) / sigma_logM))

# Recalculate HOD
N_cen = N_central(M, Mmin, sigma_logM)
N_sat = N_satellite(M, M1_val, alpha, N_cen)
N_total = N_cen + N_sat

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(log10_M, N_cen, label=r'Central galaxies', color='blue')
plt.plot(log10_M, N_sat, label=r'Satellite galaxies', color='green')
plt.plot(log10_M, N_total, label=r'Total $\langle N_{\mathrm{gal}} \rangle$', color='black', linestyle='--')

plt.xlabel(r'$\log_{10}(M_{\mathrm{halo}}/M_\odot)$')
plt.ylabel(r'$\langle N_{\mathrm{gal}} \rangle$')
plt.title('Halo Occupation Distribution (HOD)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
